# BBO Weekly Planner (GP + Acquisition + Heuristics)

This notebook gives you a **reproducible** way to generate *next-week candidate queries* for the Imperial BBO capstone.

## Important honesty note
In our chat, many week-to-week suggestions were **heuristic + pattern-based** (trust-region rollbacks, freezing plateaus, monotonic pushes), not the output of a single formal predictor.

To make this useful (and to document the methodology), this notebook implements:
1) **Gaussian Process (GP) surrogate modelling** per function (scikit-learn)
2) **Acquisition functions** (Expected Improvement, UCB)
3) A **heuristic mode** that mirrors the capstone-style reasoning.


In [ ]:
import re
import numpy as np
import pandas as pd

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel

np.set_printoptions(suppress=True, precision=6)


## 1. Parsing your capstone history files

This parser handles common formats:
- `F3: 0.930000-0.620000-0.880000`
- or plain vector lines (8 lines per week)

Adjust `extract_fx_lines()` if your files differ.


In [ ]:
FUNC_NAMES = [f"F{i}" for i in range(1, 9)]
DIMS = {"F1":2,"F2":2,"F3":3,"F4":4,"F5":4,"F6":5,"F7":6,"F8":8}

def parse_vector(s: str) -> np.ndarray:
    s = s.strip()
    parts = [p for p in s.split("-") if p]
    return np.array([float(p) for p in parts], dtype=float)

def extract_fx_lines(text: str):
    out = {k: [] for k in FUNC_NAMES}
    pat = re.compile(r"\b(F[1-8])\b\s*[:=]?\s*([0-9]\.[0-9]{6}(?:-[0-9]\.[0-9]{6}){1,7})")
    for m in pat.finditer(text):
        out[m.group(1)].append(m.group(2))
    return out

def load_history(inputs_path: str, outputs_path: str):
    with open(inputs_path, "r", encoding="utf-8") as f:
        inp_text = f.read()
    with open(outputs_path, "r", encoding="utf-8") as f:
        out_text = f.read()

    inp_map = extract_fx_lines(inp_text)
    out_map = extract_fx_lines(out_text)

    # Fallback: assume pure vector lines repeating F1..F8
    if all(len(v) == 0 for v in inp_map.values()):
        inp_lines = [ln.strip() for ln in inp_text.splitlines() if ln.strip()]
        vec_lines = [ln for ln in inp_lines if re.match(r"^[0-9]\.[0-9]{6}(?:-[0-9]\.[0-9]{6})+$", ln)]
        for i, ln in enumerate(vec_lines):
            fx = FUNC_NAMES[i % 8]
            inp_map[fx].append(ln)

    if all(len(v) == 0 for v in out_map.values()):
        out_lines = [ln.strip() for ln in out_text.splitlines() if ln.strip()]
        vec_lines = [ln for ln in out_lines if re.match(r"^[0-9]\.[0-9]{6}(?:-[0-9]\.[0-9]{6})+$", ln)]
        for i, ln in enumerate(vec_lines):
            fx = FUNC_NAMES[i % 8]
            out_map[fx].append(ln)

    history = {}
    for fx in FUNC_NAMES:
        Xs = [parse_vector(v) for v in inp_map[fx]]
        ys = []
        for raw in out_map[fx]:
            vec = parse_vector(raw)
            ys.append(float(vec[0]))
        n = min(len(Xs), len(ys))
        if n == 0:
            history[fx] = (np.zeros((0, DIMS[fx])), np.zeros((0,)))
        else:
            X = np.vstack(Xs[:n])
            y = np.array(ys[:n], dtype=float)
            history[fx] = (X, y)
    return history


## 2. GP surrogate + acquisition (EI/UCB)

We fit a GP per function and choose the next point by maximising an acquisition function.
Optimisation is done via random search over the unit hypercube.


In [ ]:
def fit_gp(X: np.ndarray, y: np.ndarray, seed: int = 0) -> GaussianProcessRegressor:
    y_mean = y.mean() if len(y) else 0.0
    y_std = y.std() if len(y) > 1 else 1.0
    y_std = y_std if y_std > 1e-8 else 1.0
    y_norm = (y - y_mean) / y_std

    kernel = ConstantKernel(1.0, (1e-3, 1e3)) * Matern(length_scale=np.ones(X.shape[1]), nu=2.5)              + WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-8, 1e-1))

    gp = GaussianProcessRegressor(
        kernel=kernel,
        alpha=0.0,
        normalize_y=False,
        n_restarts_optimizer=5,
        random_state=seed
    )
    gp.fit(X, y_norm)
    gp._y_mean = y_mean
    gp._y_std = y_std
    return gp

def gp_predict(gp: GaussianProcessRegressor, Xcand: np.ndarray):
    mu_norm, std_norm = gp.predict(Xcand, return_std=True)
    mu = mu_norm * gp._y_std + gp._y_mean
    std = std_norm * gp._y_std
    return mu, std

def ucb(mu: np.ndarray, std: np.ndarray, beta: float = 2.0):
    return mu + beta * std

def expected_improvement(mu: np.ndarray, std: np.ndarray, best: float, xi: float = 0.01):
    # Avoid SciPy dependency: approximate normal CDF/PDF via error function.
    # Good enough for ranking candidates in random search.
    import numpy as np
    sqrt2 = np.sqrt(2.0)
    Z = np.divide(mu - best - xi, std, out=np.zeros_like(mu), where=std > 1e-12)
    pdf = (1.0 / np.sqrt(2*np.pi)) * np.exp(-0.5 * Z**2)
    cdf = 0.5 * (1.0 + np.erf(Z / sqrt2))
    imp = mu - best - xi
    ei = imp * cdf + std * pdf
    ei[std <= 1e-12] = 0.0
    return ei

def propose_next_gp(X: np.ndarray, y: np.ndarray, acquisition: str = "EI",
                    n_candidates: int = 15000, seed: int = 0, beta: float = 2.0, xi: float = 0.01):
    d = X.shape[1]
    rng = np.random.default_rng(seed)

    if len(y) < max(3, d+1):
        x_next = rng.random(d)
        return x_next, {"reason":"too_few_points_random"}

    gp = fit_gp(X, y, seed=seed)
    Xcand = rng.random((n_candidates, d))
    mu, std = gp_predict(gp, Xcand)
    best = float(np.max(y))

    if acquisition.upper() == "EI":
        score = expected_improvement(mu, std, best=best, xi=xi)
    else:
        score = ucb(mu, std, beta=beta)

    idx = int(np.argmax(score))
    return Xcand[idx], {"pred_mu": float(mu[idx]), "pred_std": float(std[idx]), "best_so_far": best, "acq": acquisition}


## 3. Heuristic proposer (mirrors our weekly reasoning)

- Freeze flat/plateau functions
- Roll back after large regressions
- Otherwise do a small trust-region perturbation around the best point


In [ ]:
def propose_next_heuristic(X: np.ndarray, y: np.ndarray, step: float = 0.01, seed: int = 0):
    d = X.shape[1]
    rng = np.random.default_rng(seed)

    if len(y) == 0:
        return rng.random(d), {"reason":"no_data_random"}

    y_range = float(np.max(y) - np.min(y))
    best_idx = int(np.argmax(y))
    x_best = X[best_idx].copy()
    y_best = float(y[best_idx])

    if y_range < 1e-6:
        return np.full(d, 0.5), {"reason":"flat_freeze_center", "best": y_best}

    if len(y) >= 2 and (y_best - float(y[-1])) > 0.25 * max(1e-6, y_range):
        return x_best, {"reason":"rollback_to_best", "best": y_best}

    x = x_best.copy()
    k = 1 if d <= 3 else 2
    dims = rng.choice(d, size=k, replace=False)
    for j in dims:
        x[j] = np.clip(x[j] + rng.choice([-1.0, 1.0]) * step, 0.0, 0.999999)
    return x, {"reason":"trust_region_perturb", "best": y_best}


## 4. Generate next-week suggestions

Update these paths to your current `inputs.txt` and `outputs.txt` and run.


In [ ]:
INPUTS_PATH = "inputs.txt"
OUTPUTS_PATH = "outputs.txt"

history = load_history(INPUTS_PATH, OUTPUTS_PATH)

def fmt_vec(x):
    return "-".join([f"{v:.6f}" if v < 1 else "0.999999" for v in x])

rows = []
for fx in FUNC_NAMES:
    X, y = history[fx]
    if X.shape[0] == 0:
        X = np.zeros((0, DIMS[fx]))
        y = np.zeros((0,))

    x_gp, info_gp = propose_next_gp(X, y, acquisition="EI", n_candidates=20000, seed=42+hash(fx)%1000, xi=0.01)
    step = 0.005 if DIMS[fx] >= 4 else 0.01
    x_h, info_h = propose_next_heuristic(X, y, step=step, seed=7+hash(fx)%1000)

    rows.append({
        "Function": fx,
        "n_points": int(len(y)),
        "best_so_far": float(np.max(y)) if len(y) else np.nan,
        "GP_next": fmt_vec(x_gp),
        "GP_pred_mu": info_gp.get("pred_mu", np.nan),
        "GP_pred_std": info_gp.get("pred_std", np.nan),
        "Heuristic_next": fmt_vec(x_h),
        "Heuristic_reason": info_h.get("reason","")
    })

df = pd.DataFrame(rows)
df


## 5. Practical tips

- For **high dimensions (F7/F8)**, GP predictions can be noisy with small data. Consider using the heuristic mode or lock-in best-known points.
- For final rounds, a defensible policy is often: **submit the best observed point** (confirmation > speculation).
